In [1]:
import os
import math
import random
import regex as re
import pandas as pd
import numpy as np
from typing import Optional, Sequence

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, f1_score

from tqdm import tqdm
import torch
from torch import nn
from torch import Tensor
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer, EarlyStoppingCallback, AutoModel, AutoConfig, T5TokenizerFast, T5ForConditionalGeneration, ElectraTokenizerFast, ElectraModel

import gc
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
train = pd.read_csv('train.csv').drop(['ID'], axis=1)
test = pd.read_csv('test.csv')

In [3]:
train

,문장,유형,극성,시제,확실성,label
0,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
...,...,...,...,...,...,...
16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,사실형,긍정,과거,불확실,사실형-긍정-과거-불확실


In [4]:
train.유형.unique(), train.극성.unique(), train.시제.unique(), train.확실성.unique()

(array(['사실형', '추론형', '예측형', '대화형'], dtype=object),
 array(['긍정', '부정', '미정'], dtype=object),
 array(['현재', '과거', '미래'], dtype=object),
 array(['확실', '불확실'], dtype=object))

In [5]:
train['문장'] = train['문장'].apply(lambda x: re.sub("[^ A-Za-z0-9가-힣]", "", x))
train['문장'] = train['문장'].apply(lambda x: re.sub("[ +]", " ", x))

test['문장'] = test['문장'].apply(lambda x: re.sub("[^ A-Za-z0-9가-힣]", "", x))
test['문장'] = test['문장'].apply(lambda x: re.sub("[ +]", " ", x))

In [6]:
train['문장']

0                       075포인트 금리 인상은 1994년 이후 28년 만에 처음이다
1        이어 앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정이라며 그 이전이라도 ...
2        정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30에서 37까지 확대한다
3        서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만 하루 만에 차...
4                  익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다
                               ...                        
16536    신동덤은 신비한 동물사전과 해리 포터 시리즈를 잇는 마법 어드벤처물로 전편에 이어 ...
16537    수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목 어깨 팔꿈치 등 허...
16538    김금희 소설가는 계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 고...
16539    1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...
16540                                           목민심서의 내용이다
Name: 문장, Length: 16541, dtype: object

# Text Aug

In [7]:
# train, X_val, _, _ = train_test_split(train, train.label, test_size=0.1, random_state=42)

In [8]:
# https://github.com/catSirup/KorEDA/blob/master/eda.py
def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    return new_words

def text_aug(sentence, alpha_rs = 0.1, num_aug=3):
    words = sentence.split(' ')
    words = [word for word in words if word != ""]
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = num_aug

    n_rs = max(1, int(alpha_rs*num_words))

    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(" ".join(a_words))

    augmented_sentences = [sentence for sentence in augmented_sentences]
    random.shuffle(augmented_sentences)

    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
    return augmented_sentences

aug = train['문장'].apply(lambda x: text_aug(x))

In [9]:
tmp1 = train.copy()
tmp1['문장'] = list(map(lambda x: x[0], aug))

tmp2 = train.copy()
tmp2['문장'] = list(map(lambda x: x[1], aug))

tmp3 = train.copy()
tmp3['문장'] = list(map(lambda x: x[2], aug))

In [10]:
train = pd.concat([train,tmp1,tmp2,tmp3]).drop_duplicates(keep='first').sample(frac=1).reset_index(drop=True)
train

,문장,유형,극성,시제,확실성,label
0,전국 더욱 뽑는 비례대표 국회의원이 따로 규정된 것까지 감안하면 국회의원이 국민 대...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,우선 최근 소비자들 있는 재조명되고 사이에서 모델은 기본형에 해당하는 S20이다,사실형,긍정,현재,확실,사실형-긍정-현재-확실
2,무협 관계자는 안전운임도 부담이 나날이 증가하지만 기업들은 할증으로 운동장이어서 부...,사실형,부정,과거,확실,사실형-부정-과거-확실
3,그는 2020년까지 남북한이 통일되지 않을 경우를 상상하기는 매우 어렵다고 했다,사실형,부정,과거,확실,사실형-부정-과거-확실
4,요 몇 년 옛 할리우드에서 새 히트작이 자꾸 리메이크된 것은 신박한 이야기의 고갈을...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
...,...,...,...,...,...,...
64667,노인일자리 비판은 60만명이 넘는 빈곤노인을 그냥 방치하자는 것이냐,대화형,긍정,현재,불확실,대화형-긍정-현재-불확실
64668,트위터 역사상 가장 높은 빈도로 공유된 이 뉴스들은 합병이 끝나자 바로 사라졌다,사실형,긍정,과거,확실,사실형-긍정-과거-확실
64669,우곡은 다시 태어나더라도 누룩 개발에 전념하겠다는 뜻으로 평생을 누룩과 술 개발에 ...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
64670,고대일록의 저자 정경운도 전염병에 딸을 잃는다,사실형,긍정,현재,확실,사실형-긍정-현재-확실


In [11]:
train['문장'].str.len().max(), test['문장'].str.len().max()

(496, 378)

# Dataset

In [12]:
device = torch.device("cuda")
model_path = "kykim/electra-kor-base"
if model_path == 'paust/pko-t5-large':
    tokenizer = T5TokenizerFast.from_pretrained(model_path)
elif model_path == 'kykim/electra-kor-base':
    tokenizer = ElectraTokenizerFast.from_pretrained(model_path)
else:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
length = train['문장'].str.len().max()

In [13]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            st_type = self.labels['type'][idx]
            st_polarity = self.labels['polarity'][idx]
            st_tense = self.labels['tense'][idx]
            st_certainty = self.labels['certainty'][idx]
            item["labels"] = torch.tensor(st_type), torch.tensor(st_polarity), torch.tensor(st_tense), torch.tensor(st_certainty)
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

# HuggingFace Phase

## config

In [14]:
config=AutoConfig.from_pretrained(model_path)
config._name_or_path = 'kr.kim'
print(config.num_hidden_layers)
# config.num_hidden_layers = 17
config

12


ElectraConfig {
  "_name_or_path": "kr.kim",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

## custom model

In [15]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        if model_path == 'monologg/kobigbird-bert-base':
            config.attention_type = "original_full"
            self.base_model = AutoModel.from_pretrained(model_path, config=config)
        elif model_path == 'kykim/electra-kor-base':
            self.base_model = ElectraModel.from_pretrained(model_path, config=config)
        elif model_path == 'paust/pko-t5-large':
            self.base_model = T5ForConditionalGeneration.from_pretrained(model_path, config=config)
        else:
            self.base_model = AutoModel.from_pretrained(model_path, config=config)
        try:
            self.out = self.base_model.encoder.layer[-1].output.dense.out_features
        except:
            self.out = 768
        # self.linear = nn.Linear(768, 768//2)

        self.type_classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=self.out, out_features=4),
        )
        self.polarity_classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=self.out, out_features=3),
        )
        self.tense_classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=self.out, out_features=3),
        )
        self.certainty_classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=self.out, out_features=2),
        )
        
    def forward(self, input_ids, attention_mask, labels=None, token_type_ids=None):
        if model_path == 'paust/pko-t5-large':
            x = self.base_model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=input_ids)[0]
            # x = self.linear(x)
            # 문장 유형, 극성, 시제, 확실성을 각각 분류
            type_output = self.type_classifier(x[:,0,:].view(-1,self.out))
            polarity_output = self.polarity_classifier(x[:,0,:].view(-1,self.out))
            tense_output = self.tense_classifier(x[:,0,:].view(-1,self.out))
            certainty_output = self.certainty_classifier(x[:,0,:].view(-1,self.out))
            return type_output, polarity_output, tense_output, certainty_output
        else:
            x = self.base_model(input_ids=input_ids, attention_mask=attention_mask)[0]
            # x = self.linear(x)
            # 문장 유형, 극성, 시제, 확실성을 각각 분류
            type_output = self.type_classifier(x[:,0,:].view(-1,self.out))
            polarity_output = self.polarity_classifier(x[:,0,:].view(-1,self.out))
            tense_output = self.tense_classifier(x[:,0,:].view(-1,self.out))
            certainty_output = self.certainty_classifier(x[:,0,:].view(-1,self.out))
            return type_output, polarity_output, tense_output, certainty_output

## arg

In [16]:
# Trainer arguments
lr = 1e-4
stop = 3
epoch = 1000
batch = 32
seed = 42

## loss

In [17]:
class FocalLoss(nn.Module):
    """ Focal Loss, as described in https://arxiv.org/abs/1708.02002.
    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.
    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    """

    def __init__(self,
                 alpha: Optional[Tensor] = None,
                 gamma: float = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
        """Constructor.
        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        """
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v!r}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return torch.tensor(0.)
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss


def focal_loss(alpha: Optional[Sequence] = None,
               gamma: float = 0.,
               reduction: str = 'mean',
               ignore_index: int = -100,
               device='cpu',
               dtype=torch.float32) -> FocalLoss:
    """Factory function for FocalLoss.
    Args:
        alpha (Sequence, optional): Weights for each class. Will be converted
            to a Tensor if not None. Defaults to None.
        gamma (float, optional): A constant, as described in the paper.
            Defaults to 0.
        reduction (str, optional): 'mean', 'sum' or 'none'.
            Defaults to 'mean'.
        ignore_index (int, optional): class label to ignore.
            Defaults to -100.
        device (str, optional): Device to move alpha to. Defaults to 'cpu'.
        dtype (torch.dtype, optional): dtype to cast alpha to.
            Defaults to torch.float32.
    Returns:
        A FocalLoss object
    """
    if alpha is not None:
        if not isinstance(alpha, Tensor):
            alpha = torch.tensor(alpha)
        alpha = alpha.to(device=device, dtype=dtype)

    fl = FocalLoss(
        alpha=alpha,
        gamma=gamma,
        reduction=reduction,
        ignore_index=ignore_index)
    return fl
        
def compute_metrics(pred):
    # label = [[cls1,cls2,...],]
    # preds = n list
    focal_loss = FocalLoss()
    labels = pred.label_ids
    preds = pred.predictions
    f1 = []
    focal = []
    for i in range(4):
        # focal.append(focal_loss(torch.tensor(preds[i], dtype=torch.float), torch.tensor(labels[::, i],dtype=torch.float)))
        f1.append(f1_score(y_true = labels[::, i], y_pred = preds[i], average='weighted'))
    return {
        #'focal': sum(focal),
        'f1-sum': sum(f1)/4
    }

## trainer

In [19]:
# Define trainer
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        # forward pass
        labels = inputs.pop("labels").to(torch.int64)
        
        type_logit, polarity_logit, tense_logit, certainty_logit = model(**inputs)
        
        # # simple loss
        # criterion = {
        #     'type' : nn.CrossEntropyLoss().to(device),
        #     'polarity' : nn.CrossEntropyLoss().to(device),
        #     'tense' : nn.CrossEntropyLoss().to(device),
        #     'certainty' : nn.CrossEntropyLoss().to(device)
        # }
        # loss = criterion['type'](type_logit, labels[::, 0]) + \
        #             criterion['polarity'](polarity_logit, labels[::, 1]) + \
        #             criterion['tense'](tense_logit,labels[::, 2]) + \
        #             criterion['certainty'](certainty_logit, labels[::, 3])
        
        # focal loss
        criterion = {
            'type' : FocalLoss().to(device),
            'polarity' : FocalLoss().to(device),
            'tense' : FocalLoss().to(device),
            'certainty' : FocalLoss().to(device)
        }
        # labels = labels.type(torch.float).clone().detach()
        loss = criterion['type'](type_logit, labels[::, 0]) + \
                    criterion['polarity'](polarity_logit, labels[::, 1]) + \
                    criterion['tense'](tense_logit, labels[::, 2]) + \
                    criterion['certainty'](certainty_logit, labels[::, 3])

        outputs = None, \
                    torch.argmax(type_logit, dim = 1), \
                    torch.argmax(polarity_logit, dim = 1),\
                    torch.argmax(tense_logit, dim = 1),\
                    torch.argmax(certainty_logit, dim = 1)
        return (loss, outputs) if return_outputs else loss

# Fold

In [20]:
유형 = LabelEncoder()
유형.fit(train['유형'])

극성 = LabelEncoder()
극성.fit(train['극성'])

시제 = LabelEncoder()
시제.fit(train['시제'])

확실성 = LabelEncoder()
확실성.fit(train['확실성'])

def encoding(X_train, X_val):
    X_train['유형'] = 유형.transform(X_train['유형'])
    X_val['유형'] = 유형.transform(X_val['유형'])

    X_train['극성'] = 극성.transform(X_train['극성'])
    X_val['극성'] = 극성.transform(X_val['극성'])

    X_train['시제'] = 시제.transform(X_train['시제'])
    X_val['시제'] = 시제.transform(X_val['시제'])

    X_train['확실성'] = 확실성.transform(X_train['확실성'])
    X_val['확실성'] = 확실성.transform(X_val['확실성'])

    train_labels = {
        'type' : X_train['유형'].values,
        'polarity' : X_train['극성'].values,
        'tense' : X_train['시제'].values,
        'certainty' : X_train['확실성'].values
    }

    val_labels = {
        'type' : X_val['유형'].values,
        'polarity' : X_val['극성'].values,
        'tense' : X_val['시제'].values,
        'certainty' : X_val['확실성'].values
    }
    return train_labels, val_labels

In [21]:
config=AutoConfig.from_pretrained(model_path)
config._name_or_path = 'kr.kim'
print(f'hidden_layers : {config.num_hidden_layers}')
# config.num_hidden_layers = 10
print(f'now_hidden_layers : {config.num_hidden_layers}')

kf = KFold(n_splits=5, random_state=seed, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(train)):
    print(f'Round {i}')
    X_train, X_val = train.loc[train_index, :], train.loc[test_index, :]
    train_labels, val_labels = encoding(X_train, X_val)
    token_train, token_val = tokenizer(X_train.문장.tolist(), padding=True, truncation=True, max_length=length), tokenizer(X_val.문장.tolist(), padding=True, truncation=True, max_length=length)
    train_dataset, val_dataset = CustomDataset(token_train, train_labels), CustomDataset(token_val, val_labels)
    model = CustomModel()
    model.to(device)
    args = TrainingArguments(run_name = f'fold_{i}',                                # 모델이름
                             output_dir= f"fold_{i}",                               # 모델저장경로
                             evaluation_strategy="steps",                           # 모델의 평가를 언제 진행할지
                             eval_steps=100,                                        # 500 스텝 마다 모델 평가
                             save_steps=100,                                        # 500 스텝 마다 모델 저장
                             save_total_limit = 2,                                  # 저장할 모델의 갯수
                             logging_steps=100,                                     # 학습로스 로깅
                             per_device_train_batch_size=batch,                     # GPU에 학습데이터를 몇개씩 올려서 학습할지
                             per_device_eval_batch_size=batch,                      # GPU에 학습데이터를 몇개씩 올려서 평가할지
                             gradient_accumulation_steps=16,                        # 가상배치
                             num_train_epochs=epoch,                                # 전체 학습 진행 횟수
                             learning_rate=lr,                                      # 학습률 정의 
                             seed=seed,                                             # seed
                             load_best_model_at_end=True,                           # 평가기준 스코어가 좋은 모델만 저장할지 여부
                             fp16=True,
                             do_train=True,
                             do_eval=True,
                             # metric_for_best_model
                             # greater_is_better = True,
    )
    trainer = CustomTrainer(model=model,
                            args=args,                                                        # args
                            train_dataset=train_dataset,                                      # 학습데이터
                            eval_dataset=val_dataset,                                         # validation 데이터
                            compute_metrics=compute_metrics,                                  # 모델 평가 방식
                            callbacks=[EarlyStoppingCallback(early_stopping_patience=stop)],) # callback
    trainer.train()
    del model
    del trainer
    gc.collect() # python 자원 관리 
    torch.cuda.empty_cache() # gpu 자원관리   

hidden_layers : 12
now_hidden_layers : 12
Round 0


Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using cuda_amp half precision backend
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implem

Step,Training Loss,Validation Loss,F1-sum
100,1.251100,0.777527,0.927921
200,0.624600,0.497119,0.958422
300,0.331600,0.276576,0.979196
400,0.186500,0.189410,0.986462
500,0.112200,0.114410,0.992350
600,0.075300,0.090559,0.993984
700,0.054800,0.072712,0.995261
800,0.043800,0.074409,0.995361
900,0.032300,0.062706,0.996478
1000,0.032500,0.066702,0.996060


***** Running Evaluation *****
  Num examples = 12935
  Batch size = 32
Saving model checkpoint to fold_0/checkpoint-100
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [fold_0/checkpoint-1800] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 12935
  Batch size = 32
Saving model checkpoint to fold_0/checkpoint-200
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [fold_0/checkpoint-2100] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 12935
  Batch size = 32
Saving model checkpoint to fold_0/checkpoint-300
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [fold_0/checkpoint-100] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 12935
  Batch size = 32
Saving model checkpoint to fold_0/checkpoint-400
Trainer.model is not a `PreTrainedModel`, only saving its state dic

...

***** Running Evaluation *****
  Num examples = 12934
  Batch size = 32
Saving model checkpoint to fold_3/checkpoint-100
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [fold_3/checkpoint-1100] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 12934
  Batch size = 32
Saving model checkpoint to fold_3/checkpoint-200
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [fold_3/checkpoint-1400] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 12934
  Batch size = 32
Saving model checkpoint to fold_3/checkpoint-300
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [fold_3/checkpoint-100] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 12934
  Batch size = 32
Saving model checkpoint to fold_3/checkpoint-400
Trainer.model is not a `PreTrainedModel`, only saving its state dic

Round 4


loading weights file pytorch_model.bin from cache at /home/ubuntu/.cache/huggingface/hub/models--kykim--electra-kor-base/snapshots/8599418d72f5dcb21ae3972ba2405f88c819b195/pytorch_model.bin
Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of ElectraModel were initialized from t

Step,Training Loss,Validation Loss,F1-sum
100,1.166800,0.758433,0.932125
200,0.571700,0.462762,0.959458
300,0.305600,0.233456,0.982886
400,0.151200,0.139465,0.990809
500,0.087700,0.113452,0.992920
600,0.063800,0.093237,0.994653
700,0.051800,0.092186,0.993983
800,0.039500,0.065252,0.996192
900,0.033600,0.062386,0.996598
1000,0.032900,0.063877,0.996740


***** Running Evaluation *****
  Num examples = 12934
  Batch size = 32
Saving model checkpoint to fold_4/checkpoint-100
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [fold_4/checkpoint-1400] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 12934
  Batch size = 32
Saving model checkpoint to fold_4/checkpoint-200
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [fold_4/checkpoint-1700] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 12934
  Batch size = 32
Saving model checkpoint to fold_4/checkpoint-300
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [fold_4/checkpoint-100] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 12934
  Batch size = 32
Saving model checkpoint to fold_4/checkpoint-400
Trainer.model is not a `PreTrainedModel`, only saving its state dic

# Predict

In [22]:
def recent_file(path):
    file_name_and_time_lst = []
    # 해당 경로에 있는 파일들의 생성시간을 함께 리스트로 넣어줌. 
    for f_name in os.listdir(f"{path}"):
        written_time = os.path.getctime(f"{path}/{f_name}")
        file_name_and_time_lst.append((f_name, written_time))
    # 생성시간 역순으로 정렬하고, 
    sorted_file_lst = sorted(file_name_and_time_lst, key=lambda x: x[1], reverse=True)
    # 가장 앞에 이는 놈을 넣어준다.
    recent_file = sorted_file_lst[0]
    recent_file_name = recent_file[0]
    return f"{path}/{recent_file_name}"

In [23]:
gc.collect() # python 자원 관리 
torch.cuda.empty_cache() # gpu 자원관리
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenized = tokenizer(test.문장.tolist(), padding=True, truncation=True, max_length=length, return_tensors="pt")
test_dataset = CustomDataset(tokenized, None)
test_args = TrainingArguments(
    output_dir = './',
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 512,   
    dataloader_drop_last = False    
)

tmp = 0
while os.path.isdir(f'fold_{tmp}'):
    tmp += 1

test_results = []
for i in range(tmp):
    print(f'Round {i}')
    # model = AutoModel.from_pretrained(recent_file('custom_model'), config=config)
    model = CustomModel().to(device)
    model.load_state_dict(torch.load(f"{recent_file(f'fold_{i}')}/pytorch_model.bin"))
    trainer = CustomTrainer(
                  model = model, 
                  args = test_args, 
                  compute_metrics = compute_metrics)
    test_results.append(trainer.predict(test_dataset))
    del model
    del trainer
    gc.collect() # python 자원 관리 
    torch.cuda.empty_cache() # gpu 자원관리

loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--kykim--electra-kor-base/snapshots/8599418d72f5dcb21ae3972ba2405f88c819b195/config.json
Model config ElectraConfig {
  "_name_or_path": "kykim/electra-kor-base",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

loading file vocab.txt from ca

Round 0


Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of ElectraModel were initialized from the model checkpoint at kykim/electra-kor-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ElectraModel for predictions without furthe

loading weights file pytorch_model.bin from cache at /home/ubuntu/.cache/huggingface/hub/models--kykim--electra-kor-base/snapshots/8599418d72f5dcb21ae3972ba2405f88c819b195/pytorch_model.bin


Round 1


...

Round 3


Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of ElectraModel were initialized from the model checkpoint at kykim/electra-kor-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ElectraModel for predictions without furthe

loading weights file pytorch_model.bin from cache at /home/ubuntu/.cache/huggingface/hub/models--kykim--electra-kor-base/snapshots/8599418d72f5dcb21ae3972ba2405f88c819b195/pytorch_model.bin


Round 4


Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of ElectraModel were initialized from the model checkpoint at kykim/electra-kor-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ElectraModel for predictions without furthe

In [24]:
test['유형'] = list(map(lambda x : 유형.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions[0], test_results)))/len(test_results)))
test['극성'] = list(map(lambda x : 극성.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions[1], test_results)))/len(test_results)))
test['시제'] = list(map(lambda x : 시제.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions[2], test_results)))/len(test_results)))
test['확실성'] = list(map(lambda x : 확실성.inverse_transform([np.argmax(x)]), sum(list(map(lambda x: x.predictions[3], test_results)))/len(test_results)))

test['유형'] = list(map(lambda x : x[0], test['유형']))
test['극성'] = list(map(lambda x : x[0], test['극성']))
test['시제'] = list(map(lambda x : x[0], test['시제']))
test['확실성'] = list(map(lambda x : x[0], test['확실성']))

In [25]:
test

,ID,문장,유형,극성,시제,확실성
0,TEST_0000,장욱진의 가족은 허물 없는 가족애를 처음 공개되는 정약용의 정효자전과 정부인전은 강...,사실형,긍정,현재,확실
1,TEST_0001,조지 W 부시 버락 오바마 전 대통령도 전쟁 위험 때문에 버린 카드다,사실형,긍정,현재,확실
2,TEST_0002,지난해 1분기 128억원이었던 영업이익이 올해 1분기 505억원으로 급증했다,사실형,긍정,과거,확실
3,TEST_0003,수상 작가와 맺으려던 계약서 내용 가운데 일부가 독소 조항으로 해석돼 수정을 요청받...,사실형,긍정,과거,확실
4,TEST_0004,결국 최근 KDB산업은행은 대규모 손실 위기에 닥친 에어부산에 140억원 금융지원을...,사실형,긍정,과거,확실
...,...,...,...,...,...,...
7085,TEST_7085,2020 세계국가편람 모바일 앱은 세계 216개국의 국가개황과 주요 경제지표 사회개...,사실형,긍정,현재,확실
7086,TEST_7086,탈세계화 징후들이 반갑지 않은 이유다,추론형,긍정,현재,확실
7087,TEST_7087,틱톡은 6월 인터넷 안전의 달을 맞아 올바른 개인정보 보호 관리 방법 앱 내 유용한...,사실형,긍정,현재,확실
7088,TEST_7088,만약 3개월 간 채굴자들의 투표를 거쳐 23 이상의 해시파워가 채굴세 도입에 찬성한...,추론형,긍정,미래,확실


In [26]:
test['label'] = test['유형'] + '-' + test['극성'] + '-' + test['시제'] + '-' + test['확실성']
test

,ID,문장,유형,극성,시제,확실성,label
0,TEST_0000,장욱진의 가족은 허물 없는 가족애를 처음 공개되는 정약용의 정효자전과 정부인전은 강...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,TEST_0001,조지 W 부시 버락 오바마 전 대통령도 전쟁 위험 때문에 버린 카드다,사실형,긍정,현재,확실,사실형-긍정-현재-확실
2,TEST_0002,지난해 1분기 128억원이었던 영업이익이 올해 1분기 505억원으로 급증했다,사실형,긍정,과거,확실,사실형-긍정-과거-확실
3,TEST_0003,수상 작가와 맺으려던 계약서 내용 가운데 일부가 독소 조항으로 해석돼 수정을 요청받...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,TEST_0004,결국 최근 KDB산업은행은 대규모 손실 위기에 닥친 에어부산에 140억원 금융지원을...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
...,...,...,...,...,...,...,...
7085,TEST_7085,2020 세계국가편람 모바일 앱은 세계 216개국의 국가개황과 주요 경제지표 사회개...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
7086,TEST_7086,탈세계화 징후들이 반갑지 않은 이유다,추론형,긍정,현재,확실,추론형-긍정-현재-확실
7087,TEST_7087,틱톡은 6월 인터넷 안전의 달을 맞아 올바른 개인정보 보호 관리 방법 앱 내 유용한...,사실형,긍정,현재,확실,사실형-긍정-현재-확실
7088,TEST_7088,만약 3개월 간 채굴자들의 투표를 거쳐 23 이상의 해시파워가 채굴세 도입에 찬성한...,추론형,긍정,미래,확실,추론형-긍정-미래-확실


In [27]:
sub = pd.read_csv('sample_submission.csv')
sub['label'] = test['label']
tmp = 0
while os.path.exists(f'제출{tmp}.csv'):
    tmp += 1
sub.to_csv(f'제출{tmp}.csv', index=False, mode='w')